In [3]:
import torch
import timm
import cv2
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from torchvision import models as md
from sklearn.metrics import cohen_kappa_score, accuracy_score
from scipy.stats import trim_mean

In [5]:
base_image_dir = os.path.join('..', 'input/aptos2019-blindness-detection/')
train_dir = os.path.join(base_image_dir,'train_images/')

In [6]:
df = pd.read_csv(os.path.join(base_image_dir,"train.csv"))
df['id_code'] = df['id_code'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x)))

In [7]:
df.head()

,id_code,diagnosis
0,../input/aptos2019-blindness-detection/train_i...,2
1,../input/aptos2019-blindness-detection/train_i...,4
2,../input/aptos2019-blindness-detection/train_i...,1
3,../input/aptos2019-blindness-detection/train_i...,0
4,../input/aptos2019-blindness-detection/train_i...,0


In [8]:
submit = pd.read_csv(os.path.join(base_image_dir,"sample_submission.csv"))
submit.head()

,id_code,diagnosis
0,0005cfc8afb6,0
1,003f0afdcd15,0
2,006efc72b638,0
3,00836aaacf06,0
4,009245722fa4,0


In [9]:
def crop_image(img, tol=6):
  if img.ndim ==2:
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]
  elif img.ndim==3:
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = gray_img>tol

    check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
    if (check_shape == 0): # image is too dark so that we crop out everything,
        return img # return original image
    else:
        img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
        img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
        img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
        img = np.stack([img1,img2,img3],axis=-1)
    return img

In [10]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(24)

In [11]:
def image_resize(image, side = None, inter = cv2.INTER_AREA):
    dim = None
    h, w = image.shape[:2]
    if h < w:
        r = side/ float(h)
        dim = (int(w * r), side)
    else:
        r = side / float(w)
        dim = (side ,int(h * r))
    return cv2.resize(image, dim, interpolation = inter)

def predict(X, coef=[0.7, 1.3, 2.7, 3.3]):
    X_p = np.copy(X)
    for i, pred in enumerate(X_p):
        if pred < coef[0]:
            X_p[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            X_p[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            X_p[i] = 2
        elif pred >= coef[2] and pred < coef[3]:
            X_p[i] = 3
        else:
            X_p[i] = 4
    return np.int8(X_p) 

In [12]:
def process(i):
    img=cv2.imread('../input/aptos2019-blindness-detection/test_images/'+submit.id_code.values[i]+'.png')
    img=crop_image(img)
    img_resized=image_resize(img,300)
    cv2.imwrite("test_new/"+submit.id_code.values[i]+".png", img_resized)

In [13]:
!mkdir test_new
with concurrent.futures.ProcessPoolExecutor() as executor:
    i=range(0,len(submit))
    executor.map(process,i)  

In [14]:
def qk(y_pred, y):
    return torch.tensor(cohen_kappa_score(predict(y_pred), y, weights='quadratic'), device='cuda:0')
def accuracy(y_pred, y):
    return torch.tensor(accuracy_score(predict(y_pred), y), device='cuda:0')

In [15]:
sz=300
bs=32
tfms = get_transforms(do_flip=True,flip_vert=True, max_rotate=360, max_zoom=1.5,max_lighting=0.2)
data = (ImageList.from_df(df=df,path='',cols='id_code') 
        .split_by_rand_pct()
        .label_from_df(cols='diagnosis', label_cls=FloatList) 
        .transform(tfms,size=sz, padding_mode='border', resize_method=ResizeMethod.SQUISH)
        .databunch(bs=bs,num_workers=4) 
        .normalize(imagenet_stats)  
        )

In [16]:
import timm
learn = cnn_learner(data, timm.models.tf_efficientnet_b3,pretrained=False, metrics=[qk, accuracy],model_dir="models")
learn.data.add_test(ImageList.from_df(submit, 'test_new', suffix='.png',
                                    cols='id_code'))

In [17]:
pred=np.zeros((20, len(submit)))

In [19]:
k=10
for i in [1,2,3,4,5,6,7,8,9,10]:
    for j in range(2,4):
        name = "b3_stage-2-"+str(j)+"-"+str(i)
        print(name)
        learn.load(name)
        preds, y = learn.get_preds(DatasetType.Test)
        pred[k]=preds.squeeze()
        k+=1

In [20]:
pred.sort(axis=0)
pred_trim=trim_mean(pred, proportiontocut=0.2, axis=0)

In [21]:
tst_pred = predict(pred_trim)
submit.diagnosis = tst_pred.astype(int)
print(submit.diagnosis.value_counts())
submit.to_csv("submission.csv", index=False)

2    1282
0     334
1     122
3     118
4      72
Name: diagnosis, dtype: int64


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 